# IMPORTS

In [3]:
import os
import tempfile

from configparser import ConfigParser
os.environ["TF_USE_LEGACY_KERAS"]="1"

from tf_keras import losses,optimizers, models
import tensorflow as tf
import tensorflow_model_optimization as tfmot

In [4]:
from utils import load_dataset, reformat_image, entropy_prune_model,time_benchmark, normalize_img

In [5]:
train_examples, validation_examples, num_examples, num_classes, class_names = load_dataset("tf_flowers", 70)


INFO:absl:Generating dataset tf_flowers (C:\Users\neyen\tensorflow_datasets\tf_flowers\3.0.1)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

INFO:absl:Downloading http://download.tensorflow.org/example_images/flower_photos.tgz into C:\Users\neyen\tensorflow_datasets\downloads\tf_flowers\downl.tenso.org_examp_image_flowe_photok5_wYA8b0X3dMSAtkkUlVGs8dfM5H3yrzHcr_bVE4Ns.tgz.tmp.75569bc68c0a416585449b0c35edfd21...


Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\neyen\tensorflow_datasets\tf_flowers\incomplete.ZXKQ0Y_3.0.1\tf_flowers-train.tfrecord*...:…

INFO:absl:Done writing C:\Users\neyen\tensorflow_datasets\tf_flowers\incomplete.ZXKQ0Y_3.0.1\tf_flowers-train.tfrecord*. Number of examples: 3670 (shards: [1835, 1835])
INFO:absl:Creating a tf.data.Dataset reading 2 files located in folders: C:\Users\neyen\tensorflow_datasets\tf_flowers\3.0.1.


Dataset tf_flowers downloaded and prepared to C:\Users\neyen\tensorflow_datasets\tf_flowers\3.0.1. Subsequent calls will reuse this data.


INFO:absl:Creating a tf.data.Dataset reading 1 files located in folders: C:\Users\neyen\tensorflow_datasets\tf_flowers\3.0.1.
INFO:absl:Constructing tf.data.Dataset tf_flowers for split ('train[:70%]', 'train[70%:]'), from C:\Users\neyen\tensorflow_datasets\tf_flowers\3.0.1


In [6]:
train_examples

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [7]:
len(train_examples)

2569

In [8]:
len(validation_examples)

1101